<a href="https://colab.research.google.com/github/Simranjeet-Singh-1313/Jakarta-Ambulance-Service-Time-series-and-forecasting/blob/main/Time_Series_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install arch

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import scipy, os
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.tsa.arima_process import ArmaProcess


from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller

from arch.unitroot import PhillipsPerron
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing

from arch import arch_model
from scipy.stats import shapiro
from scipy.stats import probplot
from statsmodels.stats.diagnostic import het_arch
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Set Base-directory 
basedir='/content/drive/MyDrive/Timeseries.Assignment'

## Preprocessing part

In [ ]:
df=pd.read_excel(os.path.join(basedir,'TimeSeriesCourseworkData20_21.xlsx'))
df

In [ ]:
df["Call_Date"]=[d.date() for d in df['Call']]
df["Call_Time"]=[d.time() for d in df["Call"]]
df=df.drop("Call",axis=1)
df

In [ ]:
df["Number_of_Calls"]=df["Call_Date"].map(df.groupby("Call_Date").size())
df=df.drop_duplicates("Call_Date")
df.head(5)

In [ ]:
df.tail(10)

In [ ]:
df=df.drop(["City Municipality","Call_Time"],axis=1)
df

In [ ]:
df.tail(10)

In [ ]:
df=df.set_index("Call_Date")
df.head()

In [ ]:
df

In [ ]:
# remove first value of data which is outlier 
df=df[1:]
df

In [ ]:
df.describe()

In [ ]:
df

## Define function

In [ ]:
#helper function to plot the time series data
def plot_pred(pred,title):
    plt.figure(figsize=(17,5))
    plt.plot(train_data,label='Train')
    plt.plot(test_data,label='Actual')
    plt.plot(pred,label='Predicted')
    plt.ylabel("Jakarta Ambulance Prediction")
    plt.title(title,fontsize=20)
    plt.legend()
    plt.show()

In [ ]:
# Function Check_data check what percentage of data lies in the range mean +_ 3*sigma
def check_data(x):
    sigma = x.std()
    mean = x.mean()
    c = 0 
    for i in range(len(x)):
        if x[i] > (mean -3*sigma) and x[i] < (mean + 3*sigma):
            c = c + 1
    per_data = (c/len(x))*100

    print(f'Percentage of data between mean-3*sigma and mean+3*sigma is : {per_data}')
    

In [ ]:

#define function for ADF test
def Adf_test(series):
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(series, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p_value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
    print(f'Result: The series is {"not " if dftest[1] > 0.05 else ""}stationary')
          

In [ ]:
#Define function for KPSS test
def kpss_test(series, **kw):    
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    # Format Output
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    print(f'Result: The series is {"not " if p_value < 0.05 else ""}stationary')

In [ ]:
# Define function for PP test
def pp_test(series):    
    pp = PhillipsPerron(series)
    print(pp.summary().as_text())

In [ ]:
import math
# WE will check if there is some missing value
df = df.Number_of_Calls
miss = []
for j in range(len(df)):
    if math.isnan(df[j])== True:
        miss.append(j)
print(miss)

In [ ]:
#Plot Decomposition Figure
decomposed=sm.tsa.seasonal_decompose(df,model="additive",freq=6)
figure = decomposed.plot()
plt.figure(figsize=(18,15))
plt.show()

In [ ]:
# PLOT the given time series data
plt.figure(figsize=(15, 7))
plt.plot(df)
plt.title('Number of Calls each Day')
plt.grid(True)
plt.show()

In [ ]:
#split dataset into training set and testing set
train_data = df[:-45]
test_data = df[-45:]

# print(train_data)
# print(test_data)

In [ ]:
# PLOT the Train and Test Data in a single plot
plt.figure(figsize=(15, 7))
plt.plot(train_data.index, train_data)
plt.plot(test_data.index, test_data)
plt.title('Jakarta Ambulance Dataset')
plt.grid(True)
plt.show()

## Linear Regression

In [ ]:
df=pd.DataFrame(df)

In [ ]:
X=np.arange(df.index.size)
fit=np.polyfit(X,df.Number_of_Calls,deg=1)
print("slope :"+str(fit[0]))
print("Intercept :"+str(fit[1]))


In [ ]:
fit_Function=np.poly1d(fit)

In [ ]:
Lin_Reg_Pre=fit_Function(X)

In [ ]:
Pre_Date=pd.date_range(start="2019-01-01",end="2019-05-31",freq="D")
Pre_Date=pd.DataFrame(Pre_Date)
Pre_Date.columns=['Date']
Pre_Date.set_index(['Date'],inplace=True)
Pre_Date["Number_of_Calls"]=Lin_Reg_Pre


In [ ]:
MSE=mean_squared_error(Pre_Date,df)
RMSE=np.sqrt(MSE)
MAE=mean_absolute_error(Pre_Date,df)
print("The Mean Squar error of Linear Regression model is: ",MSE)
print("The Root Mean Squre error of Linear Regression Model is:",RMSE)
print("The Mean Absolute error of Linear Regression Model is:",MAE)

In [ ]:
prediction = fit_Function(df.index.size + 7)
print("Prediction of Linear Regression:",prediction)

In [ ]:
pri=list(np.repeat(prediction,7))

In [ ]:
Prediction_Date=pd.date_range(start="2019-06-01",end="2019-06-07",freq="D")
Prediction_Date=pd.DataFrame(Prediction_Date)
Prediction_Date.columns=['Date']
Prediction_Date.set_index(['Date'],inplace=True)
Prediction_Date["Number_of_Calls"]=pri
Prediction_Date

In [ ]:
plt.plot(df.index,fit_Function(X))
plt.plot(df.index,df.Number_of_Calls,)
plt.plot(Prediction_Date)
plt.xlabel('Date')
plt.ylabel('Number Of Calls')
plt.title('Linear Regression')
plt.show()

## Mean Method




In [ ]:
# Mean of previous values

y_av = pd.Series(np.repeat(train_data.mean(),45),index=test_data.index)
MSE=mean_squared_error(y_av,test_data)
RMSE=np.sqrt(MSE)
MAE=mean_absolute_error(y_av,test_data)
print("The Mean Squar error of Mean model model is: ",MSE)
print("The Root Mean Squre error of Mean Model is:",RMSE)
print("The Mean Absolute error of Mean Model is:",MAE)

In [ ]:
Average_pre=pd.date_range(start="2019-06-01",end="2019-06-07",freq="D")
Average_pre=pd.DataFrame(Average_pre)
Average_pre.columns=['Date']
Average_pre.set_index(['Date'],inplace=True)
mean=float(test_data.mean())
Average_pre["Number_of_Calls"]=mean
Average_pre

In [ ]:
Average_pre=Average_pre.squeeze()

In [ ]:
plt.figure(figsize=(17,5))
plt.plot(train_data,label='Train')
plt.plot(test_data,label='Actual')
plt.plot(y_av,label='Predicted')
plt.plot(Average_pre,label="Forcast")
plt.ylabel("Jakarta Ambulance Prediction")
plt.title("Mean Method",fontsize=20)
plt.legend()
plt.show()

## Moving Average

In [ ]:
# Moving Average Of Three Days
moving_Ave=test_data.rolling(3).mean()

In [ ]:
List=[179.33333,163.6667,139.3334]
for i in range(7):
  Three_day_Moving_AVG=(List[-1]+List[-2]+List[-3])/3
  List.append(Three_day_Moving_AVG)
new_list=List[3:]

In [ ]:
# Forecasting through moving Average model.
Mov_Avg_For=pd.date_range(start="2019-06-01",end="2019-06-07",freq="D")
Mov_Avg_For=pd.DataFrame(Mov_Avg_For)
Mov_Avg_For.columns=['Date']
Mov_Avg_For.set_index(['Date'],inplace=True)
Mov_Avg_For["Number_of_Calls"]=new_list
Mov_Avg_For

In [ ]:
moving_Ave.dropna(axis=0,inplace=True)

In [ ]:
MSE=mean_squared_error(moving_Ave,test_data[2:])
RMSE=np.sqrt(MSE)
MAE=mean_absolute_error(moving_Ave,test_data[2:])
print("The Mean Squar error of Moving Average model is: ",MSE)
print("The Root Mean Squre error of Moving Average Model is:",RMSE)
print("The Mean Absolute error of Moving Average Model is:",MAE)

In [ ]:
plt.figure(figsize=(17,5))
plt.plot(train_data,label='Train Data')
plt.plot(test_data,label='Test Data')
plt.plot(moving_Ave,label='Predicted')
plt.plot(Mov_Avg_For,label="Forcast")
plt.ylabel("Jakarta Ambulance Prediction")
plt.title("Moving Average",fontsize=20)
plt.legend()
plt.show()

In [ ]:

k = train_data.kurtosis()
s = train_data.skew()
print(f'Kurtosis: {k}')
print(f'skew: {s}')

In [ ]:
# Check the data Lie between Mean-3*sigma and Mean+3*sigma
check_data(train_data)

## Autocorrelation and PartialAutocorrelation


In [ ]:
plot_acf(train_data)
plt.show()

In [ ]:
plot_pacf(train_data)
plt.show()

##Detection Stationary 

In [ ]:
df=pd.DataFrame(df)

In [ ]:
Adf_test(train_data)

In [ ]:
# warnings.filterwarnings('ignore')
kpss_test(train_data)

In [ ]:
pp_test(train_data)

## Converting Series into Log


In [ ]:
#converting series into log serie to get stationary series
df['log_ret'] = np.log(df) - np.log(df.shift(1))
r_t = df['log_ret']
print(r_t)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Here we drop all NaN value
r_t = r_t.dropna()
print(r_t)

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(r_t)
plt.title('Jakarta Ambulance Time series')
plt.grid(True)
plt.show()

In [ ]:
# Break the data as test and train Data

logtrain_data = r_t[:-45]
logtest_data = r_t[-45:]
# print(logtrain_data)
# print(logtest_data)

In [ ]:
# PLot the logtrain and logtest data in a single ts graph

plt.figure(figsize=(15, 7))
plt.plot(logtrain_data.index, logtrain_data)
plt.plot(logtest_data.index, logtest_data)
plt.title('Jakarta Ambulance Dataset converted into Logrithmic series')
plt.grid(True)
plt.show()

In [ ]:
logtrain_data.describe()

In [ ]:
k = logtrain_data.kurtosis()
s = logtrain_data.skew()
print(f'Kurtosis: {k}')
print(f'skew: {s}')

In [ ]:
check_data(logtrain_data)

In [ ]:
plot_acf(logtrain_data)
plt.show()

In [ ]:

plot_pacf(logtrain_data)
plt.show()

In [ ]:
df_new=pd.DataFrame(logtrain_data)
# df_new

## Stationary Detection After converting  Log series


In [ ]:
Adf_test(logtrain_data)

In [ ]:
kpss_test(logtrain_data)

In [ ]:
pp_test(logtrain_data)

## ARIMA Model

In [ ]:
# for paprameter selection 
# Input m and n are the maximum possible value of parameter p and q respectively.
def par_sel (m,n):
    AB = {}
    for p in range(m):
        for q in range(n):
            try:
                mod = ARIMA(logtrain_data, order= (p,0,q))
                result = mod.fit()
                AB[(p,q)] = [result.aic, result.bic]
            except:
                continue
    return AB

In [ ]:
import warnings
warnings.filterwarnings("ignore")
par_sel (5,5)

In [ ]:

# FIT appropriate ARIMA model
p = 4
q = 2
mod = ARIMA(logtrain_data, order =(p,0,q))
result = mod.fit()
print(result.aic)

# Compare AIC and BIC value for p = 2 and q = 4
print(result.aic, result.bic)

In [ ]:
#Forecast on logtest_data
history = [x for x in logtrain_data]
predictions = list()
for t in range(len(logtest_data)):
    model= ARIMA(history, order=(4,0,2))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = logtest_data[t]
    history.append(obs)
    # print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test_data, predictions)
print('Test MSE: %.3f' % error)

In [ ]:
model_fit.summary()

In [ ]:
fin_pred = [train_data[-1]]
# print(fin_pred)
for i in range(len(test_data)):
    x = fin_pred[-1]
    y = predictions[i]
    fin_pred.append(x*math.exp(y))
fin_pred = fin_pred[1:]

In [ ]:
# for j in range(len(fin_pred)):
    # print(fin_pred[j],":", test_data[j])

In [ ]:
MSE= mean_squared_error(fin_pred, test_data)
RMSE=np.sqrt(MSE)
MAE=mean_absolute_error(fin_pred, test_data)
print("The Mean Squar error of ARIMA model is: ",MSE)
print("The Root Mean Squre error of ARIMA Model is:",RMSE)
print("The Mean Absolute error of ARIMA Model is:",MAE)


In [ ]:
# for paprameter selection 
# Input m and n are the maximum possible value of parameter p and q respectively.
def par_sel (m,n):
    AB = {}
    for p in range(m):
        for q in range(n):
            try:
                mod = ARIMA(logtest_data, order= (p,0,q))
                result = mod.fit()
                AB[(p,q)] = [result.aic, result.bic]
            except:
                continue
    return AB

In [ ]:
import warnings
warnings.filterwarnings("ignore")
par_sel (5,5)

In [ ]:
history_1=[x for x in logtest_data]
forcast_ARIMA=ARIMA(history_1,order=(2,0,1)).fit()
forcast1=forcast_ARIMA.forecast(7)

In [ ]:
forcast_ARIMA.summary()

In [ ]:
Arima_for=pd.date_range(start="2019-06-01",end="2019-06-07",freq="D")
Arima_for=pd.DataFrame(Arima_for)
Arima_for.columns=['Date']
Arima_for.set_index(['Date'],inplace=True)
# Arima_for

In [ ]:
Number_of_Calls=list()
for i in list(range(len(Arima_for))):
  X=forcast1[0][i]
  # print(type(X))
  Number_of_Calls.append(X)
# Number_of_Calls

In [ ]:
final_prediction = [test_data[-1]]
for i in range(len(Arima_for)):
    x = final_prediction[-1]
    y = Number_of_Calls[i]
    final_prediction.append(x*math.exp(y))
final_prediction= final_prediction[1:]

In [ ]:
# print Dataframe for Prediction Of Number Of Calls by ARIMA 
Arima_for["Number_of_Calls"]=final_prediction
Arima_for

In [ ]:
print(df.Number_of_Calls.mean(),np.sqrt(df.Number_of_Calls.var()))

In [ ]:
test_data=pd.DataFrame(test_data)
test_data
new_df=test_data.drop("Number_of_Calls",axis=1)
fin_pred=pd.DataFrame(fin_pred)
# fin_pred

In [ ]:
new_df.index
fin_pred.index=new_df.index
fin_pred.columns=["Number_of_predicted_Calls"]

In [ ]:
#Plot Arima Model
plt.figure(figsize=(17,5))
plt.plot(train_data,label='Train Data')
plt.plot(test_data,label='Test Data')
plt.plot(fin_pred,label='Predicted')
plt.plot(Arima_for,label="Forcast")
plt.ylabel("Jakarta Ambulance Prediction")
plt.title("ARIMA Model",fontsize=20)
plt.legend()
plt.show()

# HOLT Winters

In [ ]:
# Holt Winter Model fit on training set
model=ExponentialSmoothing(logtrain_data,trend='additive',seasonal='add',seasonal_periods=29).fit()

In [ ]:
# test_forcast=holt_win_model.forecast(46)
test_forcast=model.predict(start="2019-04-17",end="2019-05-31")

In [ ]:
final_forcast=[train_data[-1]]
for i in range(len(test_forcast)):
  x=final_forcast[-1]
  y=test_forcast[i]
  final_forcast.append(x*math.exp(y))
holt_forcast1=pd.DataFrame(final_forcast)
holt_forcast1["Call_Date"]=pd.date_range(start="2019-04-16",end="2019-05-31",freq="D")
holt_forcast1.index=holt_forcast1["Call_Date"]

In [ ]:
MSE = mean_squared_error(final_forcast[1:], test_data)
RMSE=np.sqrt(MSE)
MAE=mean_absolute_error(final_forcast[1:], test_data)
print("The Mean Squar error of Holt Winter model is: ",MSE)
print("The Root Mean Squre error of Holt Winter Model is:",RMSE)
print("The Mean Absolute error of Holt Winter Model is:",MAE)

In [ ]:
holt_forcast1.drop(["Call_Date"],inplace=True,axis=1)

In [ ]:
holt_win_model=ExponentialSmoothing(logtest_data,trend='additive',seasonal='additive',seasonal_periods=7).fit()

In [ ]:
test_pred1=holt_win_model.predict(start="2019-06-01",end="2019-06-07")

In [ ]:
final_pred=[test_data.Number_of_Calls[-1]]
for i in range(len(test_pred1)):
  x=final_pred[-1]
  y=test_pred1[i]
  final_pred.append(x*math.exp(y))
holt_pre=pd.DataFrame(final_pred)
holt_pre["Call_Date"]=pd.date_range(start="2019-06-01",periods=8,freq="D")
# holt_pre.index=test_pred.index

In [ ]:
holt_pre=holt_pre[:7]
holt_pre=holt_pre.set_index("Call_Date")
holt_pre=holt_pre.set_axis(['Number_of_Calls'], axis='columns')
holt_pre=holt_pre.set_axis(['Number_of_Calls'], axis='columns')

In [ ]:
# Print DataFrame of Holt Winter Prediction Number Of Calls
holt_pre

In [ ]:
# Plot Holt Winter Method
plt.figure(figsize=(17,5))
plt.plot(train_data,label='Train Data')
plt.plot(test_data,label='Test Data')
plt.plot(holt_forcast1,label='prediction')
plt.plot(holt_pre,label='Forcast')
plt.ylabel("Jakarta Ambulance Prediction")
plt.title("Holt Winters",fontsize=20)
plt.legend()
plt.show()